In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import youtubecollector as ytc
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [3]:
youtube_client = ytc.youtube_client.create_youtube_client("./api.conf")

Google Developer Api key: ········


In [4]:
channel_seed_filename = "input/seeds_right_extra.csv"
channel_outputfile = "output/right/channels_right_extra.csv"

In [6]:
channel_seed_df = pd.read_csv(channel_seed_filename)

channels = ytc.channels.get_channels(channel_seed_df.loc[0:], youtube_client)

ytc.channels.write_channels(channels, channel_outputfile)

UC16j6EppP0K85CzYMduNCqw


In [7]:
video_output_file = "output/right/videos_roosh.csv"

In [8]:
all_videos = list()

for channel in tqdm(channels):
    response = ytc.video.get_videos(channel.channel_uploads, youtube_client)
    next_page_token = response.get('nextPageToken')
    videos = ytc.video.convert_to_videos(response, youtube_client)
    all_videos.extend(videos)
    ytc.video.write_videos(videos, video_output_file)
    
    while next_page_token:
        response = ytc.video.get_more_videos(channel.channel_uploads, youtube_client, next_page_token)            
        next_page_token = response.get('nextPageToken')
        videos = ytc.video.convert_to_videos(response, youtube_client)
        all_videos.extend(videos)
        ytc.video.write_videos(videos, video_output_file)

In [11]:
comments_output_file = "output/right/comments_roosh.csv"

In [12]:

all_comments = list()
for video in tqdm(all_videos):
    response = ytc.comments.get_comments(video.video_id, youtube_client)
    comments = ytc.comments.convert_to_comments(response)
    all_comments.extend(comments)
    ytc.comments.write_comments(comments_output_file, comments)
    try:
        next_page_token = response.get('nextPageToken')
    except AttributeError:
        continue
    
    while next_page_token:
        response = ytc.comments.get_more_comments(video.video_id, youtube_client, next_page_token)
        try:
            next_page_token = response.get('nextPageToken')                       
        except AttributeError:
            continue
        comments = ytc.comments.convert_to_comments(response)
        all_comments.extend(comments)
        ytc.comments.write_comments(comments_output_file, comments)

In [13]:
recommendations_output_file = "output/right/recommendations_roosh.csv"


In [15]:
video_to_recommendations = dict()
for video in tqdm(all_videos, ):

    try:
        response = ytc.recommendations.get_recommendations(video.video_id, youtube_client)
    except:
        continue
    
    recommendations = ytc.recommendations.convert_to_recommendations(response, video.video_id)
    video_to_recommendations[video.video_id]=recommendations
    
    ytc.recommendations.write_recommendations(recommendations_output_file, recommendations)

In [16]:
transcripts_output_file = "/home/dim/Documents/projecten/extremisme/youtube/yt/YouTubeExtremism/DataCollection/output/right/transcripts_roosh.csv"

In [17]:
ytc.transcripts.get_captions(all_videos)


[generic] Society']): Requesting header
[redirect] Following redirect to https://www.youtube.com/watch?v=video(video
[generic] watch?v=video(video: Requesting header


[generic] watch?v=video(video: Downloading webpage
[generic] watch?v=video(video: Extracting information


ERROR: Unsupported URL: https://www.youtube.com/watch?v=video(video


[generic] Society']): Requesting header
[redirect] Following redirect to https://www.youtube.com/watch?v=video(video
[generic] watch?v=video(video: Requesting header


[generic] watch?v=video(video: Downloading webpage
[generic] watch?v=video(video: Extracting information


...

[generic] watch?v=video(video: Downloading webpage
[generic] watch?v=video(video: Extracting information


ERROR: Unsupported URL: https://www.youtube.com/watch?v=video(video


[generic] Lifestyle_(sociology)']): Requesting header
[redirect] Following redirect to https://www.youtube.com/watch?v=video(video
[generic] watch?v=video(video: Requesting header


[generic] watch?v=video(video: Downloading webpage
[generic] watch?v=video(video: Extracting information


ERROR: Unsupported URL: https://www.youtube.com/watch?v=video(video


In [18]:
video_id_transcripts = ytc.transcripts.extract_transcripts("./*.vtt")

ytc.transcripts.write_transcripts(transcripts_output_file, video_id_transcripts)

In [19]:
import os
import shutil
import glob

for file in glob.glob('*.vtt'):
    try:
        shutil.move(file, 'output/right/raw_data/transcripts_right/')
    except:
        os.remove(file)

In [44]:
df = pd.read_csv('output/right/comments_roosh.csv', sep='¶',
                        quotechar='þ',
                engine='python')
df.head()

,video_id,comment_id,author_display_name,author_channel_url,author_channel_id,comment_text,comment_like_count,comment_dislike_count,comment_time,reply_count
0,P2yvLgWLCS0,Ugzulmgzl2DI-Q7nbr14AaABAg,SuperHantz,http://www.youtube.com/channel/UCX4aRY-GscOoa0...,UCX4aRY-GscOoa05delEcoXg,THX 1138,0,0.0,2018-12-04T07:54:51.000Z,0.0
1,P2yvLgWLCS0,Ugwuio49p2gbkkPLEft4AaABAg,Ashesofexistance,http://www.youtube.com/channel/UCd5jZUVBr0foMj...,UCd5jZUVBr0foMjjKl2__-ew,das juden is known for pushing moral degenerac...,0,0.0,2018-12-04T07:32:40.000Z,0.0
2,P2yvLgWLCS0,UgwYJVFEnuruz-3W_QV4AaABAg,E A,http://www.youtube.com/channel/UCqWkjRWVoESxpB...,UCqWkjRWVoESxpB4_f3e9mvQ,Your left eye is way more open than your right...,0,0.0,2018-12-04T06:17:23.000Z,0.0
3,P2yvLgWLCS0,UgxlleBf9IOvJ_9syUF4AaABAg,Meera Venkatesan,http://www.youtube.com/channel/UC2_AgaJqmWDzBQ...,UC2_AgaJqmWDzBQhoIya2Arg,reported this video harassment,0,0.0,2018-12-04T03:59:51.000Z,0.0
4,P2yvLgWLCS0,Ugxnfeywv__CsAiCvtR4AaABAg,Miguel Cruz,http://www.youtube.com/channel/UCR9r6j7fFxUg-3...,UCR9r6j7fFxUg-3wb6ewsnLQ,read &quot;The Wanting Seed&quot; by Anthony B...,0,0.0,2018-12-04T03:00:56.000Z,0.0


In [45]:
df.to_csv('output/right/comments_right.csv', mode='a')

In [34]:
df = pd.read_csv('output/right/videos_right.csv', encoding='latin-1')


ParserError: Error tokenizing data. C error: Expected 16 fields in line 262950, saw 17


In [35]:
df = pd.read_csv('output/right/comments_right.csv', nrows=1)

In [36]:
df

,¶video_id¶comment_id¶author_display_name¶author_channel_url¶author_channel_id¶comment_text¶comment_like_count¶comment_dislike_count¶comment_time¶reply_count
0,0¶fyan-kaRZIs¶UgxpYIazGqyoSPexD554AaABAg¶The g...
